In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# show all outputs of a cell (such as if df.head() and df.tail() are in the same cell)
#default is 'last_expr'

%matplotlib inline
import pandas as pd
import sklearn
import pickle
import numpy as np
from pandasql import sqldf
import matplotlib.pyplot as plt

sql = lambda q: sqldf(q, globals()) 

product=[('B','1121','PC'),('A','1232','PC'),('A','1233','PC'),('E','1260','PC'),('A','1276','Printer'),\
   ('D','1288','Printer'),('A','1298','Laptop'),('C','1321','Laptop'),('A','1401','Printer'),('A','1408','Printer'),\
   ('D','1433','Printer'),('E','1434','Printer'),('B','1750','Laptop'),('A','1752','Laptop'),('E','2113','PC'),\
   ('E','2112','PC')]
product=pd.DataFrame(product,columns=['maker','model','type'])

pc=[(1,'1232',500,64,5,'12x',600),(2,'1121',750,128,14,'40x',850),(3,'1233',500,64,5,'12x',600),\
(4,'1121',600,128,14,'40x',850),(5,'1121',600,128,8,'40x',850),(6,'1233',750,128,20,'50x',950),\
(7,'1232',500,32,10,'12x',400),(8,'1232',450,64,8,'24x',350),(9,'1232',450,32,10,'24x',350),\
(10,'1260',500,32,10,'12x',350),(11,'1233',900,128,40,'40x',980),(12,'1233',800,128,20,'50x',970)]
pc=pd.DataFrame(pc,columns=['code','model','speed','ram','hd','cd','price'])           

laptop=[(1,'1298',350,32,4,700,11),(2,'1321',500,64,8,970,12),(3,'1750',750,128,12,1200,14),\
(4,'1298',600,64,10,1050,15),(5,'1752',750,128,10,1150,14),(6,'1298',450,64,10,950,12)]
laptop=pd.DataFrame(laptop,columns=['code','model','speed','ram','hd','price','screen'])

printer=[(1,'1276','n','Laser',400),(2,'1433','y','Jet',270),(3,'1434','y','Jet',290),\
(4,'1401','n','Matrix',150),(5,'1408','n','Matrix',270),(6,'1288','n','Laser',400)]
printer=pd.DataFrame(printer,columns=['code','model','color','type','price'])

PC [code, model, speed, ram, hd, cd, price] <br>
product [maker, model, type]<br>
laptop [code, model, speed, ram, hd, price, screen]<br>
printer [code, model, color, type, price]

join key: model (this is not unique in laptop table)

 <b> Exercise: 8 (Serge I: 2003-02-03) Find the makers producing PCs but not laptops

In [225]:
q='''
with 
temp1 as (select maker from product p join laptop l on l.model = p.model group by maker),
temp2 as (select maker from product p join pc on pc.model = p.model group by maker)

select maker from product 
where maker not in (select maker from temp1) and maker in (select maker from temp2)
group by maker

'''
sql(q)

,maker
0,E


In [184]:
q='''
select maker from product p
where p.maker not in (select maker from product p join laptop l on p.model = l.model)
      and p.type = 'PC'
group by maker
'''
sql(q)

,maker
0,E


<b> 7. Get the models and prices for all commercially available products (of any type) produced by maker B.

The below query does the following: <br>
1. Makes temp - creates maker b product table, left joins all other table information <br>
2. Makes temp2 which converts None and Nan price values to 0
3. Select statement sums temp2 price columns for each prod type into one price column

In [166]:
q='''

with temp2 as (
with temp as (
select p.maker, p.model, pr.price as pr_price, l.price as l_price, pc.price as pc_price 
from product p
left join printer pr on pr.model = p.model
left join laptop l on l.model = p.model
left join pc on pc.model = p.model
    where maker = 'B' and
    (p.model in (select model from pc)
    or p.model in (select model from laptop)
    or p.model in (select model from printer))
)
select model,
case when pr_price > 0 then pr_price else 0 end as pr_price,
case when l_price  > 0 then l_price else 0 end as l_price,
case when pc_price > 0 then pc_price else 0 end as pc_price
from temp
)

select model, pr_price+l_price+pc_price as price from temp2
group by model, price

'''
sql(q)

,model,price
0,1121,850
1,1750,1200


In [12]:
#7
q='''
select printer.model, printer.price 
from product join printer on printer.model = product.model
where maker = 'B'

union

select laptop.model, laptop.price
from product join laptop on laptop.model = product.model
where maker= 'B'

union

select pc.model, pc.price
from product join pc on pc.model = product.model
where maker= 'B'

'''
sql(q)


,model,price
0,1121,850
1,1750,1200


In [126]:
#6 
q='''
select p.maker, l.speed from laptop l
inner join product p on p.model = l.model
where l.hd>=10
order by p.maker, l.speed
'''
sql(q)

q='''
select maker, speed from laptop, product
where laptop.model = product.model and laptop.hd>=10
order by maker, speed
'''
sql(q)

,maker,speed
0,A,450
1,A,600
2,A,750
3,B,750


,maker,speed
0,A,450
1,A,600
2,A,750
3,B,750


In [74]:
#5
q='''
select model, speed, ram from pc
where price<600 and cd in ('12x','24x')
'''
sql(q)

q='''
select model, speed, ram from pc
where price<600 and (cd = '12x' or cd = '24x')
'''
sql(q)

,model,speed,ram
0,1232,500,32
1,1232,450,64
2,1232,450,32
3,1260,500,32


,model,speed,ram
0,1232,500,32
1,1232,450,64
2,1232,450,32
3,1260,500,32


In [56]:
#4
q='''
select * from printer where color = 'y'
'''
sql(q)

q='''
select p1.* from printer p1
inner join printer p2 on p1.model = p2.model
and p2.color = 'y'
'''
sql(q)

q='''
select p1.* from printer p1
inner join printer p2 on p1.model = p2.model
and p2.color = 'y'
'''
sql(q)

,code,model,color,type,price
0,2,1433,y,Jet,270
1,3,1434,y,Jet,290


,code,model,color,type,price
0,2,1433,y,Jet,270
1,3,1434,y,Jet,290


In [39]:
#3
q='''
select model, ram, screen from laptop where
price>1000
'''
sql(q)

,model,ram,screen
0,1750,128,14
1,1298,64,15
2,1752,128,14


In [38]:
#2
q='''
select p.maker from product p
inner join printer pr on pr.model = p.model
group by p.maker
'''
sql(q)

q='''
select maker from product, printer
where product.model = printer.model
group by maker
'''
sql(q)

,maker
0,A
1,D
2,E


,maker
0,A
1,D
2,E


In [4]:
#1
q='''
select model, speed, ram from pc 
where price <500
'''
sql(q)

,model,speed,ram
0,1232,500,32
1,1232,450,64
2,1232,450,32
3,1260,500,32
